# Segmentation of the customer database and behavioral profiling
## By Daria Lapko, Konstantin Shashkov, Timofey Sluev. DSBA 201
### Data Analysis in Business

### Libraries

Let's import the necessary modules first

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

ModuleNotFoundError: No module named 'matplotlib'

## Part 1. Data exploration
<hr color=green>

In [ ]:
# First, we need to read the data
df_base = pd.read_excel("data.xlsx")
df = df_base.copy(deep=True)
df.head(20)

In [ ]:
# View some data
df.head(10)

In [ ]:
# Observe some data information
df.info()

We can observe that the data presents a set of orders of an online store. It consists of 356017 entries and 22 fields, including the ones describing the product type, the customer's contacts, the order status, delivery information, and timing. Each record contains the following datatypes: 1 datetime field (*OrderDate*), 9 int64 fields, and 12 object fields, which constitute to the string data that could not be automatically parsed to any standard Pandas datatype.

### Missing values

Let us print the number of missing values accross different columns and their percentage among all data in this column.

In [ ]:
# Count the missing values
df_nan = pd.DataFrame({})
df_nan["# empty (NaN)"] = df.isna().sum()
df_nan["% empty (NaN)"] = df.isna().sum()/df[df.columns[0]].count()*100
df_nan

The printout above shows that only 64 rows out of 356017 contain missing values. The percentage on the right shows that this number is negligibly small, so one can remove such records without any significant impact on the analysis.

In [ ]:
# Drop missing values
df.dropna(inplace=True)

### Duplicated records
Every record represents an order to the online store. The field *Order_ID* should be uniquely defined for every order and vice-versa. Hence, all similar records should be dropped from the dataframe.

In [ ]:
# Drop duplicate entries
df.drop_duplicates(inplace=True)
df.info()

### Now, consider the numeric and categorical variables (as automatically predetected) separately.

In [ ]:
# Separate numerical data from categorical data
data_num = df.select_dtypes(exclude=['object', 'datetime64'])
data_cat = df.select_dtypes(include=['object', 'datetime64'])

In [ ]:
data_num.head()

In [ ]:
data_cat.head()

### Consider the categorical variables first

In [ ]:
# Review the number of categorical unique values
df_cat_info = pd.DataFrame({})
df_cat_info["# unique"] = data_cat.nunique(dropna=False)
df_cat_info["% unique"] = data_cat.nunique()/data_cat[data_cat.columns[0]].count()*100
df_cat_info

According to the table above, the highest number of unique values is found in *Order_ID* and *Phone_new* column, while *Source* and *Delivery* type have the lowest number.

Now, we shall print the values of different categorical variables to check them one by one for possible issues.

In [ ]:
# Check the unique values
uniq_vals = pd.DataFrame()
for i in data_cat.columns:
    print(i)
    print(pd.unique(data_cat[i]))

Firstly, we spotted repetitions and similar values in *CancelReason*. Let us merge them.

In [ ]:
# Group cancel reasons
df["CancelReason"] = df['CancelReason'].replace('Витринный образец/Брак товара', "Витрина/брак/некондиция/качество товара")
df["CancelReason"] = df['CancelReason'].replace('Качество товара', "Витрина/брак/некондиция/качество товара")
df["CancelReason"] = df['CancelReason'].replace('Не устроило время доставки', "Не устроили дата/время доставки")
df["CancelReason"] = df['CancelReason'].replace('Не устроила дата доставки', "Не устроили дата/время доставки")

It was also found that names of some variables are not completely understandable and do not convey the meaning of this variable. Hence, we shall rename them.

In [ ]:
# Make names sensible
df.rename(columns={"NomGroup": "Item", "TN": "Category", "TK": "Type"}, inplace=True)

Right now columns stand in not a very conveninent order. We propose to swap *Category* and *Type* (right now we consider only categorical-type columns).

In [ ]:
# Reindex dataframe
df = df.reindex(columns=df.columns)

Remove "Доставка" column. Since it is added to most of the orders and breaks the clustering.

In [ ]:
# Drop to avoid multicollinearilty
df = df.drop(df[df["Item"] == "Доставка"].index)
df

Finally, we've seen that some variables contradict each other, and so we should delete such rows so not to mess up during further analysis.

In [ ]:
# Bad Status, No Cancel Reason
df = df.drop(df[((df['Status'] == 'Отменен') | (df['Status'] == 'Отказ на месте')) & (df['CancelReason'] == 'Не отменен')].index)

# Good Status, Bad Cancel Reason
df = df.drop(df[((df['Status'] != 'Отменен') & (df['Status'] != 'Отказ на месте')) & (df['CancelReason'] != 'Не отменен')].index)

# Bad DeliveryType, No Cancel Reason
df = df.drop(df[(df['DeliveryType'] == 'Заказ отменен') & (df['CancelReason'] == 'Не отменен')].index)

# Bad DeliveryType, Good Status
df = df.drop(df[(df['DeliveryType'] == 'Заказ отменен') & ((df['Status'] != 'Отменен') & (df['Status'] != 'Отказ на месте'))].index)

For visual representation and to better understand the data contents, we build the graphs and charts of several variables.

In [ ]:
# Pie chart of ways to receive an order
plt.pie(df['DeliveryType'].value_counts(), labels=df['DeliveryType'].unique(), pctdistance=.7, autopct='%1.1f%%')
plt.title("Pie chart of ways to receive an order");

It is clearly seen from the pie chart that more than 3/4 of people chose delivery when making an order. About one fifth of all customers canceled their order, and only a very small part took self pickup.

Important note: the fact that about 20% of orders according to the chart were canceled does not mean this is the real number of orders canceled. As we can see from the initial table, there exist orders which were assigned to delivery/pickup but had status "Отменен".

In [ ]:
# Bar chart of payment types
fig, ax = plt.subplots(figsize=(17,5))
sns.barplot(y=df['PaymentType'].unique(), x=df['PaymentType'].value_counts())
plt.title("Bar chart of payment types")
plt.ylabel("Payment type")
plt.xlabel("Count")

Looking at the bar chart above we can see that most of the orders were paid by cash: more than 250 000 cases of such. The second popular way to pay is by card when receiving the order: about 50 000 cases. The least popular ways are through Tinkoff bank and PayPal system.

In [ ]:
# Countplot of orders in different categories
sns.countplot(data=df, y='Category', order=df['Category'].value_counts(ascending=False).index)
plt.title("Horizontal count plot of orders in different categories")
plt.xlabel("Count");

The last plot shows us that the greatest amount of things is bought in category "Услуги". This is simply because this category includes delivery, which is paid and goes with almost every other category, as we have seen in pie chart before. Looking at the real items, the most is bought in "Мелкая бытовая техника" (about 40 000) and "Крупная бытовая техника" (about 30 000), while the least in "Инсталляция" and "Автотовары".

In [ ]:
# Region count by category
plt.figure(figsize= (10,10))
sns.countplot(data=df, order=df["Region"].value_counts(ascending=False).index, x="Region", hue="Category", dodge=False, palette="tab20")
plt.title('Region Count by Category');

In [ ]:
# Categories in SpB and LenOblast
plot_data_lenobl = df[df['Субъект']=='Ленинградская обл.']
plot_data_spb = df[df['Субъект']=='Санкт-Петербург']
plot_data1 = pd.concat([plot_data_spb, plot_data_lenobl])
plt.figure(figsize=(10,5))
sns.countplot(data=plot_data1, y='Category', order=df["Category"].value_counts(ascending=False).index)
plt.title("Categories Bought in St. Petersburg and Leningrad Region");

In [ ]:
# Categories in Moscow
plot_data = df[df['Субъект']=='Москва']
plt.figure(figsize=(10,5))
sns.countplot(data=plot_data, y='Category', order=df["Category"].value_counts(ascending=False).index)
plt.title("Categories Bought in Moscow");

In [ ]:
# Order time distribution
sns.histplot(data = df, x = 'Время заказа в часах')
plt.xlabel("Order time")
plt.title("Order time distribution");

Unfortunately, *OrderDate* cannot be encoded as categorical variable, so we should invent another way to present time data. We decided to divide date on day number, month number and weekday. Three new columns with therefore appear instead of *OrderDate*: *dayNum*, *monthNum*, and *weekdayNum*.

In [ ]:
# Encode datetime
df["dayNum"] = df["OrderDate"].dt.day
df["monthNum"] = df["OrderDate"].dt.month
df["weekdayNum"] = df["OrderDate"].dt.weekday
date = df["OrderDate"]
df = df.drop(columns = "OrderDate")

The *Phone_new* column is deleted from the data frame because it cannot be used for modelling (is categorical but cannot be encoded normally), but its data is saved in a numeric way (frequency of purchases) for future use in RFM model.

In [ ]:
# Work with Phone_new category
countBuy = df['Phone_new'].map(df['Phone_new'].value_counts())
df = df.drop(columns = "Phone_new")

### Now consider the numeric features

In [ ]:
# Consider the numeric features
df_numeric = pd.DataFrame({})
df_numeric["mean"] = data_num.mean()
df_numeric["median"] = data_num.median()
df_numeric["min"] = data_num.min()
df_numeric["max"] = data_num.max()
df_numeric["sd"] = data_num.std()
df_numeric["# unique"] = data_num.nunique(dropna=False)
df_numeric["% unique"] = data_num.nunique()/data_num[data_num.columns[0]].count()*100
df_numeric["# zero"] = data_num.agg(lambda x: x.eq(0).sum())
df_numeric["% zero"] = data_num.agg(lambda x: x.eq(0).sum())/data_num[data_num.columns[0]].count()*100
df_numeric

The information above tells us general information about every single numeric variable. For example, we can see that median number of items bought of 1 type is 1, while maximum number is 285; the mean row price is 5123.02 while standard deviation is 10737.78; etc. We can also spot that there is a variable with binary (0 and 1) values only: *OneClick*.

In [ ]:
# Correlation heatmap
correlation_mat = data_num.corr()
plt.figure(figsize=(12,10), dpi=80)
sns.heatmap(correlation_mat, annot = True, fmt=".2f")
plt.title("Correlation matrix");

According to the correlation matrix printed above, most of the numeric variables do not correlate (have neglibile values that range from -0.10 to 0.10). High positive correlation is spotted between *Week* and *месяц*, *RowSum* and *RowPrice*; small to medium correlation is between *RowPrice* and *RowDiscount*, *RowSum* and *RowDiscount*. All these correlations are explainable: *RowSum* includes *RowDiscount* and *RowPrice*, while *месяц* and *Week* are date variables.

It was spotted that *месяц* variable is not informative since we already have *OrderDate* which contains year, month and day of order. Hence, we can delete this column from the table.

In [ ]:
# Remove multicollinearity
df = df.drop(columns="месяц")

### Product categories

Consider a table of product categories to have a better understnading of the store's assortment.


In [ ]:
product_categories = data_cat[['TN', 'TK', 'NomGroup']].drop_duplicates().sort_values(by=['TN', 'TK'])
product_categories

### Data preparation

Before moving on to models, we ought to prepare a new data frame, which will not include cofusing data for the algorithms. The columns *Order_ID* and *Store_ID* do own such data, and even though it is important in general we cannot utilize it in modelling.

In [ ]:
data = df.copy()
data = data.drop(columns=["Order_ID", "Store_ID"])
data_initial = data.copy()
data

To prepare categorical data for making models we ought to perform encoding of variables. We chose one-hot encoding because this is one of the easiest and most popular encodings, and also because no other encodings is suitable here (e.g. ordinal encoding: data in variables cannot be ordered here).

In [ ]:
categorical_cols = data.select_dtypes(include=["object"]).columns
data = pd.get_dummies(data, columns = categorical_cols)
data

## Model

#### K-means clustering

In [ ]:
data_scaled = MinMaxScaler().fit_transform(data)

In [ ]:
data_pca = PCA(n_components=0.8, random_state=42).fit_transform(data_scaled)
data_pca

In [ ]:
from sklearn.cluster import KMeans

sse = []
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
    kmeans.fit(data_pca)
    sse.append(kmeans.inertia_)

plt.plot(range(1, 10), sse)
plt.xticks(range(1, 10))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.title('Elbow Method Analysis');

In [ ]:
model = KMeans(init="random", n_clusters=5, n_init=10, random_state=1)
model.fit(data_pca)
fitted = model.labels_

In [ ]:
labels = np.unique(fitted)
centroids = model.cluster_centers_
plt.figure(figsize=(10, 10))
for i in labels:
     plt.scatter(data_pca[fitted == i , 0] , data_pca[fitted == i , 1] , label = i)
plt.scatter(centroids[:,0] , centroids[:,1] , s = 80, color = 'k')
plt.legend()
plt.title('K-Means clustering');

In [ ]:
df_numclus = pd.DataFrame({})
df_numclus["% data"] = np.unique(fitted, return_counts=True)[1]/df.shape[0]
df_numclus

#### DBSCAN

In [ ]:
data_sampled = data.sample(n=100000)

In [ ]:
data_scaled = MinMaxScaler().fit_transform(data_sampled)

In [ ]:
data_pca = PCA(n_components=0.8, random_state=42).fit_transform(data_scaled)
data_pca

In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=1.8).fit(data_pca)
np.unique(dbscan.labels_)

In [ ]:
labels = np.unique(dbscan.labels_)
plt.figure(figsize=(10, 10))
for i in labels:
     plt.scatter(data_pca[dbscan.labels_ == i , 0] , data_pca[dbscan.labels_ == i , 1] , label = i)
plt.legend()
plt.title('DBSCAN clustering');

#### BIRCH

In [ ]:
from sklearn.cluster import Birch
brc = Birch(n_clusters=3)
birch_labels = brc.fit_predict(data_pca)

In [ ]:
brcl = np.unique(birch_labels)
plt.figure(figsize=(10, 10))
for i in brcl:
     plt.scatter(data_pca[birch_labels == i , 0] , data_pca[birch_labels == i , 1] , label = i)
plt.legend()
plt.title('BIRCH clustering');

### RFM

In [ ]:
df_RFM = data.copy()
date = date.apply(lambda x : x.timestamp()).astype(int)
df_RFM["OrderDate"] = date
df_RFM["NumOrders"] = countBuy
df_RFM_overview = pd.DataFrame({})
df_RFM_overview["mean"] = round(df_RFM[["OrderDate", "NumOrders", "RowSum"]].mean(),2)
df_RFM_overview["min"] = df_RFM[["OrderDate","NumOrders", "RowSum"]].min()
df_RFM_overview["max"] = df_RFM[["OrderDate","NumOrders", "RowSum"]].max()
df_RFM_overview

In [ ]:
quantiles = round(df_RFM[["OrderDate", "NumOrders", "RowSum"]].quantile([1.0/3.0, 2.0/3.0]),0).to_dict()
quantiles

In [ ]:
def R_score(x):
    if x <= quantiles['OrderDate'][1.0/3.0]:
        return 1
    elif x <= quantiles['OrderDate'][2.0/3.0]:
        return 2
    else:
        return 3

def FM_score(x, y):
    if x <= quantiles[y][1.0/3.0]:
        return 3
    elif x <= quantiles[y][2.0/3.0]:
        return 2
    else:
        return 1

In [ ]:
df_RFM['R'] = df_RFM['OrderDate'].apply(lambda x: R_score(x))
df_RFM['F'] = df_RFM['NumOrders'].apply(lambda x: FM_score(x, 'NumOrders'))
df_RFM['M'] = df_RFM['RowSum'].apply(lambda x: FM_score(x, 'RowSum'))

In [ ]:
df_RFM['RFMScore'] = df_RFM['R'].map(str) + df_RFM['F'].map(str) + df_RFM['M'].map(str)
df_RFM['RFMScore'] = df_RFM['RFMScore'].astype(int)
df_RFM.head()

In [ ]:
fig=plt.figure(figsize=(10, 5), dpi= 80, facecolor='w', edgecolor='k')
df_RFM.groupby('RFMScore').agg('NumOrders').mean().plot(kind='bar')
plt.title("Mean Values of Recency vs RFM Score")
plt.ylabel("Secs");

In [ ]:
def RFMcluster(lower, upper):
    return df_RFM.loc[(df_RFM['RFMScore'] >= lower) & (df_RFM['RFMScore'] <= upper)]

In [ ]:
RFM_1 = RFMcluster(111,133)
RFM_2 = RFMcluster(211,233)
RFM_3 = RFMcluster(311,333)

In [ ]:
pd.concat([RFM_1["RFMScore"].mode(), RFM_2["RFMScore"].mode(), RFM_3["RFMScore"].mode()]).dropna().T

Hence, our targets for analysis are the following segments: **132**, **232**, **332**.

## Clustering interpretation

In [ ]:
len(fitted)

In [ ]:
ser = pd.Series(fitted)
data_initial = data_initial.assign(K_means = fitted)
data_initial

### K-Means clustering interpretation

In [ ]:
df_kmeans = pd.DataFrame({})
for col in data_initial.select_dtypes(exclude=['object', 'datetime64']):
    df_kmeans[col] = round(data_initial[col].groupby(data_initial["K_means"]).mean(),2)
df_kmeans = df_kmeans.drop(columns="K_means")
df_kmeans

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(8,15))
cnt = 0
for i in data_initial["DeliveryType"].unique():
    sns.countplot(data=data_initial[data_initial["DeliveryType"] == i], x="K_means", ax=ax[cnt])
    if (cnt == 0):
        ax[cnt].title.set_text('Delivery Type Count by Cluster');
    cnt+=1

In [ ]:
data_initial["DeliveryType"].unique()

In [ ]:
sns.countplot(data=data_initial, y="CancelReason")
plt.title('Cancel Reason Values Count');

In [ ]:
sns.countplot(data=data_initial[data_initial["CancelReason"] == "Не отменен"], x="K_means")
plt.title('Completed Orders Count by Cluster');

In [ ]:
sns.countplot(data=data_initial[data_initial["K_means"] == 2], order=data_initial["CancelReason"].value_counts(ascending=False).index, y="CancelReason")
plt.title('Cancel Reason Values Count in 2nd Cluster');

In [ ]:
fig, ax = plt.subplots(7, 1, figsize=(8,13))
cnt = 0
print(data_initial["PaymentType"].unique())
for i in data_initial["PaymentType"].unique():
    sns.countplot(data=data_initial[data_initial["PaymentType"] == i], x="K_means", ax=ax[cnt], label=i)
    if (cnt == 0):
        ax[cnt].title.set_text('Payment Type Count by Cluster');
    cnt+=1

In [ ]:
plt.figure(figsize= (15,15))
sns.countplot(data=data_initial, order=data_initial["Город"].value_counts(ascending=False).index, y="Город", hue="K_means", dodge=False)
plt.title('City Count by Cluster');

### RFM clustering interpretation

In [ ]:
data_initial = data_initial.assign(RFMScore = df_RFM["RFMScore"])
data_initial

In [ ]:
data_initial = data_initial.loc[(df_RFM['RFMScore'] == 132) | (df_RFM['RFMScore'] == 232) | (df_RFM['RFMScore'] == 332)]
df_RFMseg = pd.DataFrame({})
for col in data_initial.select_dtypes(exclude=['object', 'datetime64']):
  df_RFMseg[col] = round(data_initial[col].groupby(data_initial["RFMScore"]).mean(),2)
df_RFMseg = df_RFMseg.drop(columns=["K_means","RFMScore"])
df_RFMseg

In [ ]:
plt.figure(figsize= (15,15))
sns.countplot(data=data_initial, y="Region", palette='bright', hue="RFMScore", dodge=True)
plt.title('Region Count by Cluster');

In [ ]:
plt.figure(figsize= (15,15))
sns.countplot(data=data_initial, order=data_initial["Type"].value_counts(ascending=False).index, y="Type", hue="RFMScore", dodge=False)
plt.title('Type Count by Cluster');

In [ ]:
plt.figure(figsize= (15,15))
sns.countplot(data=data_initial, order=data_initial["Category"].value_counts(ascending=False).index, y="Category", hue="RFMScore", dodge=False)
plt.title('Category Count by Cluster');

In [ ]:
plt.figure(figsize= (15,15))
sns.countplot(data=data_initial, x="weekdayNum", hue="RFMScore", palette="bright")
plt.title('Week Day Count by Cluster');